In [1]:
!pip install ultralytics -q

In [2]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image

In [3]:
base_dir=Path("/kaggle/input/kitti-dataset")
img_path=base_dir / 'data_object_image_2' / 'training' / 'image_2'
label_path=Path("/kaggle/input/kitti-dataset-yolo-format/labels")

In [4]:
with open("/kaggle/input/kitti-dataset-yolo-format/classes.json","r") as f:
    classes=json.load(f)

In [5]:
classes

{'Car': 0,
 'Pedestrian': 1,
 'Van': 2,
 'Cyclist': 3,
 'Truck': 4,
 'Misc': 5,
 'Tram': 6,
 'Person_sitting': 7}

In [6]:
images=sorted(list(img_path.glob("*")))

In [7]:
labels=sorted(list(label_path.glob("*")))

In [8]:
pairs=list(zip(images,labels))
pairs[:4]

[(PosixPath('/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000000.png'),
  PosixPath('/kaggle/input/kitti-dataset-yolo-format/labels/000000.txt')),
 (PosixPath('/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000001.png'),
  PosixPath('/kaggle/input/kitti-dataset-yolo-format/labels/000001.txt')),
 (PosixPath('/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000002.png'),
  PosixPath('/kaggle/input/kitti-dataset-yolo-format/labels/000002.txt')),
 (PosixPath('/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000003.png'),
  PosixPath('/kaggle/input/kitti-dataset-yolo-format/labels/000003.txt'))]

In [9]:
train, test = train_test_split(pairs, test_size=0.2,shuffle=True, random_state=42)

In [10]:
print("Length of train set:", len(train))
print("Length of test set:", len(test))

Length of train set: 5984
Length of test set: 1497


In [11]:
train_path = Path('train').resolve()
train_path.mkdir(exist_ok=True)

In [12]:
valid_path = Path('valid').resolve()
valid_path.mkdir(exist_ok=True)

In [13]:
for t_image,t_label in tqdm(train):
    im_path=train_path / t_image.name
    lb_path=train_path / t_label.name
    shutil.copy(t_image,im_path)
    shutil.copy(t_label,lb_path)    

  0%|          | 0/5984 [00:00<?, ?it/s]

In [14]:
for t_image,t_label in tqdm(test):
    im_path=valid_path / t_image.name
    lb_path=valid_path / t_label.name
    shutil.copy(t_image,im_path)
    shutil.copy(t_label,lb_path)    

  0%|          | 0/1497 [00:00<?, ?it/s]

In [15]:
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open('kitti.yaml','w') as f:
    f.write(yaml_file)

In [16]:
!cat kitti.yaml


names:
- Car
- Pedestrian
- Van
- Cyclist
- Truck
- Misc
- Tram
- Person_sitting
nc: 8
train: /kaggle/working/train
val: /kaggle/working/valid

In [17]:
model=YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]


In [18]:
train_results=model.train(
    data="/kaggle/working/kitti.yaml",
    epochs=20,
    patience=3,
    mixup=0.1,
    project="yolov8n-kitti",
)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/kitti.yaml, epochs=20, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolov8n-kitti, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

100%|██████████| 755k/755k [00:00<00:00, 19.7MB/s]
2024-04-12 04:51:59,397	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-12 04:52:00,189	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-12 04:52:02.733605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 04:52:02.733738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 04:52:02.877573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])

In [ ]:
valid_results = model.val()


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train/labels.jpg"))
plt.axis('off')
plt.show()

In [ ]:
import os

In [ ]:
os.listdir("/kaggle/working/yolov8n-kitti/train2")

In [ ]:
for file in os.listdir("/kaggle/working/yolov8n-kitti/train2"):
    if file.startswith("val"):
        plt.figure(figsize=(20,10))
        plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/{}".format(file)))
        plt.title(file)
        plt.axis('off')
        plt.show()


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/R_curve.png"))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/P_curve.png"))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/F1_curve.png"))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/PR_curve.png"))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/confusion_matrix.png"))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Image.open("/kaggle/working/yolov8n-kitti/train2/confusion_matrix_normalized.png"))
plt.axis('off')
plt.show()

In [ ]:
len(test)

In [ ]:
preds = model.predict([test[idx][0] for idx in np.random.randint(0,len(test),(20,))],save=True)

In [ ]:
preds

In [ ]:
list(Path("/kaggle/working/yolov8n-kitti/train3/").glob("*"))

In [ ]:
preds=list(Path("/kaggle/working/yolov8n-kitti/train3/").glob("*"))

In [ ]:
preds

In [ ]:
def plot_images(images):
    num_image=len(images)
    rows=num_image
    cols=1
    fig,axes=plt.subplots(rows,cols,figsize=(15,80))
    for ax in axes:
        ax.axis("off")
    for i,img_path in enumerate(images):
        img=Image.open(img_path)
        axes[i].imshow(img)
    plt.tight_layout()
    plt.show()
    

In [ ]:
plot_images(preds)

In [ ]:
model.save("yolo8Model_Main.h5")